# Gerador de Canais Sintéticos

## Modelo de Difusão

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Configurações gerais
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Função para o agendamento linear dos betas
def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

# Função para criar embeddings de tempo
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        emb = np.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
        return emb

# Modelo U-Net aprimorado para o processo reverso com embeddings de tempo
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, n_feat=64, time_emb_dim=256):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.time_emb_dim = time_emb_dim

        # Embedding de tempo
        self.time_mlp = nn.Sequential(
            SinusoidalPosEmb(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim),
            nn.ReLU()
        )

        # Encoder
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels + time_emb_dim, n_feat, kernel_size=3, padding=1),
            nn.BatchNorm1d(n_feat),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(n_feat, n_feat * 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(n_feat * 2),
            nn.ReLU()
        )

        # Decoder
        self.conv3 = nn.Sequential(
            nn.ConvTranspose1d(n_feat * 2, n_feat, kernel_size=3, padding=1),
            nn.BatchNorm1d(n_feat),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.ConvTranspose1d(n_feat, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x, t):
        # x: (batch_size, in_channels, seq_length)
        # t: (batch_size,)

        # Incorporar o embedding de tempo
        t_emb = self.time_mlp(t)  # (batch_size, time_emb_dim)

        # Expandir t_emb para combinar com x
        t_emb = t_emb.unsqueeze(-1)  # (batch_size, time_emb_dim, 1)
        t_emb = t_emb.repeat(1, 1, x.size(-1))  # (batch_size, time_emb_dim, seq_length)

        # Concatenar o embedding de tempo com x
        x = torch.cat([x, t_emb], dim=1)  # (batch_size, in_channels + time_emb_dim, seq_length)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        return x  # Saída com shape: (batch_size, out_channels, seq_length)

# Função de treinamento ajustada
def train_diffusion_model(model, optimizer, scheduler, dataloader, timesteps, betas, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, epochs=100):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for i, (batch_inputs, batch_targets) in tqdm(enumerate(dataloader), total=len(dataloader), leave=False):
            batch_inputs = batch_inputs.to(device)     # Shape: (batch_size, num_input_channels, seq_length)
            batch_targets = batch_targets.to(device)   # Shape: (batch_size, 1, seq_length)

            batch_size = batch_inputs.size(0)
            t = torch.randint(0, timesteps, (batch_size,), device=device).long()

            # Adicionar ruído aos alvos
            noise = torch.randn_like(batch_targets)
            sqrt_alphas_cumprod_t = sqrt_alphas_cumprod[t].view(batch_size, 1, 1)
            sqrt_one_minus_alphas_cumprod_t = sqrt_one_minus_alphas_cumprod[t].view(batch_size, 1, 1)
            x_t = sqrt_alphas_cumprod_t * batch_targets + sqrt_one_minus_alphas_cumprod_t * noise  # x_t para os alvos

            # Concatenar entradas e x_t
            model_input = torch.cat([batch_inputs, x_t], dim=1)  # Shape: (batch_size, in_channels, seq_length)

            optimizer.zero_grad()
            noise_pred = model(model_input, t)

            loss = nn.MSELoss()(noise_pred, noise)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        scheduler.step()

# Função para gerar o canal sintético
def sample_from_model(model, conditioning_inputs, alphas, alphas_cumprod, betas, sqrt_one_minus_alphas_cumprod, shape):
    model.eval()
    batch_size = shape[0]
    seq_length = shape[2]

    x = torch.randn((batch_size, 1, seq_length), device=device)
    with torch.no_grad():
        for t in reversed(range(len(betas))):
            t_batch = torch.tensor([t]*batch_size, device=device).long()
            beta_t = betas[t]
            alpha_t = alphas[t]
            alpha_cumprod_t = alphas_cumprod[t]
            sqrt_one_minus_alpha_cumprod_t = sqrt_one_minus_alphas_cumprod[t]
            sqrt_alpha_t = torch.sqrt(alpha_t)

            # Concatenar conditioning_inputs e x
            model_input = torch.cat([conditioning_inputs, x], dim=1)

            noise_pred = model(model_input, t_batch)

            x = (1 / sqrt_alpha_t) * (x - (beta_t / sqrt_one_minus_alpha_cumprod_t) * noise_pred)
    return x

# Carregar os dados de EEG utilizando pandas
eeg_df = pd.read_csv('bci_competition_3_v.csv')

# Lista de diretrizes para criar canais sintéticos
diretrizes = [
    ### cenario 1
    # {'canal_alvo': 'AF3', 'canais_entrada': ['Fp1', 'F3']},
    # {'canal_alvo': 'AF4', 'canais_entrada': ['Fp2', 'F4']},
    # {'canal_alvo': 'F7', 'canais_entrada': ['FC5', 'F3']},
    # {'canal_alvo': 'F8', 'canais_entrada': ['T8', 'F4']},
    # {'canal_alvo': 'T7', 'canais_entrada': ['C3', 'CP1']},
    # {'canal_alvo': 'T8', 'canais_entrada': ['C4', 'CP2']},
    # {'canal_alvo': 'Fp1', 'canais_entrada': ['AF3', 'F3']},
    # {'canal_alvo': 'Fp2', 'canais_entrada': ['AF4', 'F4']}

    ### cenario 2
    # {'canal_alvo': 'F3', 'canais_entrada': ['AF4', 'F8']},
    {'canal_alvo': 'F4', 'canais_entrada': ['T7', 'CP1']},
    # {'canal_alvo': 'C3', 'canais_entrada': ['T8', 'CP2']},
    # {'canal_alvo': 'C4', 'canais_entrada': ['T7', 'F7']},
    # {'canal_alvo': 'CP1', 'canais_entrada': ['T7', 'P3']},
    # {'canal_alvo': 'CP2', 'canais_entrada': ['T8', 'P4']},
    # {'canal_alvo': 'P3', 'canais_entrada': ['CP1', 'O1']},
    # {'canal_alvo': 'P4', 'canais_entrada': ['CP2', 'O2']},

    ### cenario 3

    # {'canal_alvo': 'FC3', 'canais_entrada': ['C3', 'FC1']},
    # {'canal_alvo': 'FC4', 'canais_entrada': ['C4', 'CP1']},
    # {'canal_alvo': 'CP3', 'canais_entrada': ['C3', 'CP2']},
    # {'canal_alvo': 'CP4', 'canais_entrada': ['C4', 'F7']},
    # {'canal_alvo': 'Pz', 'canais_entrada': ['Cz', 'P3']},
    # {'canal_alvo': 'Fz', 'canais_entrada': ['Cz', 'FC1']},
    # {'canal_alvo': 'Oz', 'canais_entrada': ['POz', 'O1']},
    # {'canal_alvo': 'P0z', 'canais_entrada': ['Pz', 'PO3']},

]

# Configurar os parâmetros da difusão
timesteps = 1000
betas = linear_beta_schedule(timesteps).to(device)
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod).to(device)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1 - alphas_cumprod).to(device)

# Lista para armazenar as métricas de avaliação
avaliacao_resultados = []

# Iterar sobre cada diretriz
for diretriz in diretrizes:
    canal_alvo = diretriz['canal_alvo']
    canais_entrada = diretriz['canais_entrada']
    
    print(f"\nTreinando o modelo para prever o canal {canal_alvo} usando {canais_entrada}\n")
    
    # Selecionar os dados dos canais de entrada e do canal alvo
    inputs = eeg_df[canais_entrada].values
    targets = eeg_df[[canal_alvo]].values

    # Calcular a média e desvio padrão dos canais de entrada e alvo
    mean_inputs = inputs.mean(axis=0)
    std_inputs = inputs.std(axis=0)
    inputs_normalizado = (inputs - mean_inputs) / std_inputs

    mean_target = targets.mean(axis=0)
    std_target = targets.std(axis=0)
    targets_normalizado = (targets - mean_target) / std_target

    # Garantir que inputs e targets tenham o mesmo número de amostras
    min_length = min(len(inputs_normalizado), len(targets_normalizado))
    inputs_normalizado = inputs_normalizado[:min_length]
    targets_normalizado = targets_normalizado[:min_length]

    # Preparação dos dados: Segmentar em sequências menores
    seq_length = 128
    num_samples = inputs_normalizado.shape[0]
    num_segments = num_samples // seq_length

    inputs_normalizado = inputs_normalizado[:num_segments * seq_length]
    targets_normalizado = targets_normalizado[:num_segments * seq_length]

    inputs_segments = inputs_normalizado.reshape(num_segments, seq_length, len(canais_entrada))
    targets_segments = targets_normalizado.reshape(num_segments, seq_length, 1)

    # Transpor para (num_segments, num_channels, seq_length)
    inputs_segments = inputs_segments.transpose(0, 2, 1)
    targets_segments = targets_segments.transpose(0, 2, 1)

    # Criar Dataset personalizado
    class EEGDataset(torch.utils.data.Dataset):
        def __init__(self, inputs, targets):
            self.inputs = torch.tensor(inputs, dtype=torch.float32)
            self.targets = torch.tensor(targets, dtype=torch.float32)

        def __len__(self):
            return len(self.inputs)

        def __getitem__(self, idx):
            x = self.inputs[idx]
            y = self.targets[idx]
            return x, y

    # Criar o dataset e dataloader
    dataset = EEGDataset(inputs_segments, targets_segments)
    batch_size = 32
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Definir o modelo U-Net ajustado para os canais de EEG
    time_emb_dim = 128
    in_channels = len(canais_entrada) + 1  # Canais de entrada + canal alvo com ruído adicionado
    out_channels = 1  # Ruído previsto para o canal alvo
    model = UNet(in_channels=in_channels, out_channels=out_channels, n_feat=128, time_emb_dim=time_emb_dim).to(device)

    # Otimizador e scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    # Treinar o modelo
    epochs = 200
    train_diffusion_model(model, optimizer, scheduler, dataloader, timesteps, betas, sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, epochs=epochs)

    # Salvar o modelo treinado
    model_filename = f'modelo_{canal_alvo}.pth'
    torch.save(model.state_dict(), model_filename)
    print(f"Modelo salvo como {model_filename}")

    # Gerar o canal sintético para todos os segmentos
    conditioning_inputs = torch.tensor(inputs_segments, dtype=torch.float32).to(device)

    num_samples = conditioning_inputs.size(0)
    seq_length = conditioning_inputs.size(2)
    shape = (num_samples, 1, seq_length)

    # Gerar o canal sintético
    synthetic_channels = sample_from_model(model, conditioning_inputs, alphas, alphas_cumprod, betas, sqrt_one_minus_alphas_cumprod, shape=shape)

    # Desnormalizar os dados
    synthetic_channels = synthetic_channels.cpu().numpy().squeeze()  # Shape: (num_samples, seq_length)
    synthetic_channels_desnormalizado = synthetic_channels * std_target + mean_target  # Desnormalizar

    # Flatten the synthetic data
    synthetic_channel_flat = synthetic_channels_desnormalizado.reshape(-1)

    # Criar uma nova coluna no eeg_df para o canal sintético
    column_name = f'Sintetico_{canal_alvo}'

    # Garantir que o comprimento corresponda aos índices em eeg_df
    eeg_df[column_name] = np.nan  # Inicializar a nova coluna com NaNs
    eeg_df.loc[:len(synthetic_channel_flat)-1, column_name] = synthetic_channel_flat

    # Calcular o MSE e a correlação usando todos os dados
    original_target = targets_normalizado.reshape(-1) * std_target + mean_target
    synthetic_target = synthetic_channel_flat

    mse = mean_squared_error(original_target[:len(synthetic_target)], synthetic_target)
    corr = np.corrcoef(original_target[:len(synthetic_target)], synthetic_target)[0, 1]
    print(f"MSE para {canal_alvo} = {mse:.4f}, Correlação = {corr:.4f}")

    # Armazenar as métricas de avaliação
    avaliacao_resultados.append({
        'Canal_Alvo': canal_alvo,
        'Canais_Entrada': ', '.join(canais_entrada),
        'MSE': mse,
        'Correlacao': corr
    })

# Converter os resultados em um DataFrame
avaliacao_df = pd.DataFrame(avaliacao_resultados)
print("\nResultados da Avaliação:")
print(avaliacao_df)

# Salvar os resultados da avaliação em um arquivo CSV
avaliacao_df.to_csv(f'avaliacao_resultados_modelo_difusao_{canal_alvo}.csv', index=False)
print(f"Resultados da avaliação salvos em 'avaliacao_resultados_modelo_difusao_{canal_alvo}.csv'")

# Salvar o eeg_df com os dados sintéticos
eeg_df.to_csv(f'eeg_com_sintetico_modelo_difusao_{canal_alvo}.csv', index=False)
print(f"Dados sintéticos adicionados e salvos em 'eeg_com_sintetico_modelo_difusao_{canal_alvo}.csv'")



Treinando o modelo para prever o canal F3 usando ['AF4', 'F8']



Epoch 1/200, Loss: 6.5723


Epoch 2/200, Loss: 3.1714


Epoch 3/200, Loss: 2.2431


Epoch 4/200, Loss: 1.7406


...

Epoch 196/200, Loss: 0.1674


Epoch 197/200, Loss: 0.1651


Epoch 198/200, Loss: 0.1768


Epoch 199/200, Loss: 0.1648


Epoch 200/200, Loss: 0.1627
Modelo salvo como modelo_F3.pth
MSE para F3 = 6.9567, Correlação = 0.6025

Resultados da Avaliação:
  Canal_Alvo Canais_Entrada       MSE  Correlacao
0         F3        AF4, F8  6.956672    0.602463
Resultados da avaliação salvos em 'avaliacao_resultados_modelo_difusao_F3.csv'
Dados sintéticos adicionados e salvos em 'eeg_com_sintetico_modelo_difusao_F3.csv'


## Modelo de Difusão - c/ Hyperparameter Tuning e Cross-Validation - INTERESSANTE

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# Configurações gerais
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Função para o agendamento linear dos betas
def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

# Função para criar embeddings de tempo
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        emb = np.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)
        return emb

# Modelo U-Net aprimorado para o processo reverso com embeddings de tempo
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, n_feat=64, time_emb_dim=256):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.time_emb_dim = time_emb_dim

        # Embedding de tempo
        self.time_mlp = nn.Sequential(
            SinusoidalPosEmb(time_emb_dim),
            nn.Linear(time_emb_dim, time_emb_dim),
            nn.ReLU()
        )

        # Encoder
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels + time_emb_dim, n_feat, kernel_size=3, padding=1),
            nn.BatchNorm1d(n_feat),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(n_feat, n_feat * 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(n_feat * 2),
            nn.ReLU()
        )

        # Decoder
        self.conv3 = nn.Sequential(
            nn.ConvTranspose1d(n_feat * 2, n_feat, kernel_size=3, padding=1),
            nn.BatchNorm1d(n_feat),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.ConvTranspose1d(n_feat, out_channels, kernel_size=3, padding=1)
        )

    def forward(self, x, t):
        # x: (batch_size, in_channels, seq_length)
        # t: (batch_size,)

        # Incorporar o embedding de tempo
        t_emb = self.time_mlp(t)  # (batch_size, time_emb_dim)

        # Expandir t_emb para combinar com x
        t_emb = t_emb.unsqueeze(-1)  # (batch_size, time_emb_dim, 1)
        t_emb = t_emb.repeat(1, 1, x.size(-1))  # (batch_size, time_emb_dim, seq_length)

        # Concatenar o embedding de tempo com x
        x = torch.cat([x, t_emb], dim=1)  # (batch_size, in_channels + time_emb_dim, seq_length)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        return x  # Saída com shape: (batch_size, out_channels, seq_length)

# Função de treinamento ajustada
def train_diffusion_model(model, optimizer, scheduler, dataloader, timesteps, betas,
                          sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, epochs=100):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for i, (batch_inputs, batch_targets) in tqdm(enumerate(dataloader), total=len(dataloader), leave=False):
            batch_inputs = batch_inputs.to(device)     # Shape: (batch_size, num_input_channels, seq_length)
            batch_targets = batch_targets.to(device)   # Shape: (batch_size, 1, seq_length)

            batch_size = batch_inputs.size(0)
            t = torch.randint(0, timesteps, (batch_size,), device=device).long()

            # Adicionar ruído aos alvos
            noise = torch.randn_like(batch_targets)
            sqrt_alphas_cumprod_t = sqrt_alphas_cumprod[t].view(batch_size, 1, 1)
            sqrt_one_minus_alphas_cumprod_t = sqrt_one_minus_alphas_cumprod[t].view(batch_size, 1, 1)
            x_t = sqrt_alphas_cumprod_t * batch_targets + sqrt_one_minus_alphas_cumprod_t * noise  # x_t para os alvos

            # Concatenar entradas e x_t
            model_input = torch.cat([batch_inputs, x_t], dim=1)  # Shape: (batch_size, in_channels, seq_length)

            optimizer.zero_grad()
            noise_pred = model(model_input, t)

            loss = nn.MSELoss()(noise_pred, noise)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        scheduler.step()

# Função para gerar o canal sintético
def sample_from_model(model, conditioning_inputs, alphas, alphas_cumprod, betas,
                      sqrt_one_minus_alphas_cumprod, shape):
    model.eval()
    batch_size = shape[0]
    seq_length = shape[2]

    x = torch.randn((batch_size, 1, seq_length), device=device)
    with torch.no_grad():
        for t in reversed(range(len(betas))):
            t_batch = torch.tensor([t]*batch_size, device=device).long()
            beta_t = betas[t]
            alpha_t = alphas[t]
            alpha_cumprod_t = alphas_cumprod[t]
            sqrt_one_minus_alpha_cumprod_t = sqrt_one_minus_alphas_cumprod[t]
            sqrt_alpha_t = torch.sqrt(alpha_t)

            # Concatenar conditioning_inputs e x
            model_input = torch.cat([conditioning_inputs, x], dim=1)

            noise_pred = model(model_input, t_batch)

            x = (1 / sqrt_alpha_t) * (x - (beta_t / sqrt_one_minus_alpha_cumprod_t) * noise_pred)
    return x

# Carregar os dados de EEG utilizando pandas
eeg_df = pd.read_csv('bci_competition_3_v.csv')

# Lista de diretrizes para criar canais sintéticos
diretrizes = [
    {'canal_alvo': 'AF3', 'canais_entrada': ['Fp1', 'F3']},
    # {'canal_alvo': 'AF4', 'canais_entrada': ['Fp2', 'F4']},
    # {'canal_alvo': 'F7', 'canais_entrada': ['FT7', 'F3']},
    # {'canal_alvo': 'F8', 'canais_entrada': ['FT8', 'F4']},
    # {'canal_alvo': 'T7', 'canais_entrada': ['C3', 'TP7']},
    # {'canal_alvo': 'T8', 'canais_entrada': ['C4', 'TP8']},
    # {'canal_alvo': 'Fp1', 'canais_entrada': ['AF3', 'F3']},
    # {'canal_alvo': 'Fp2', 'canais_entrada': ['AF4', 'F4']}
]

# Configurar os parâmetros da difusão
timesteps = 1000
betas = linear_beta_schedule(timesteps).to(device)
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod).to(device)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1 - alphas_cumprod).to(device)

# Lista para armazenar as métricas de avaliação
avaliacao_resultados = []

# Definir hiperparâmetros para ajuste
hiperparametros = [
    {'n_feat': 64, 'lr': 1e-4, 'batch_size': 32, 'epochs': 200},
    {'n_feat': 128, 'lr': 1e-4, 'batch_size': 32, 'epochs': 200},
    {'n_feat': 128, 'lr': 5e-5, 'batch_size': 32, 'epochs': 200},
    # Adicione mais configurações conforme necessário
]

# Definir o número de folds para cross-validation
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterar sobre cada diretriz
for diretriz in diretrizes:
    canal_alvo = diretriz['canal_alvo']
    canais_entrada = diretriz['canais_entrada']
    
    print(f"\nProcessando o canal {canal_alvo} usando {canais_entrada}\n")
    
    # Selecionar os dados dos canais de entrada e do canal alvo
    inputs = eeg_df[canais_entrada].values
    targets = eeg_df[[canal_alvo]].values

    # Calcular a média e desvio padrão dos canais de entrada e alvo
    mean_inputs = inputs.mean(axis=0)
    std_inputs = inputs.std(axis=0)
    inputs_normalizado = (inputs - mean_inputs) / std_inputs

    mean_target = targets.mean(axis=0)
    std_target = targets.std(axis=0)
    targets_normalizado = (targets - mean_target) / std_target

    # Garantir que inputs e targets tenham o mesmo número de amostras
    min_length = min(len(inputs_normalizado), len(targets_normalizado))
    inputs_normalizado = inputs_normalizado[:min_length]
    targets_normalizado = targets_normalizado[:min_length]

    # Preparação dos dados: Segmentar em sequências menores
    seq_length = 128
    num_samples = inputs_normalizado.shape[0]
    num_segments = num_samples // seq_length

    inputs_normalizado = inputs_normalizado[:num_segments * seq_length]
    targets_normalizado = targets_normalizado[:num_segments * seq_length]

    inputs_segments = inputs_normalizado.reshape(num_segments, seq_length, len(canais_entrada))
    targets_segments = targets_normalizado.reshape(num_segments, seq_length, 1)

    # Transpor para (num_segments, num_channels, seq_length)
    inputs_segments = inputs_segments.transpose(0, 2, 1)
    targets_segments = targets_segments.transpose(0, 2, 1)

    # Converter para tensores
    inputs_tensor = torch.tensor(inputs_segments, dtype=torch.float32)
    targets_tensor = torch.tensor(targets_segments, dtype=torch.float32)

    # Iterar sobre as configurações de hiperparâmetros
    for config in hiperparametros:
        n_feat = config['n_feat']
        lr = config['lr']
        batch_size = config['batch_size']
        epochs = config['epochs']
        
        print(f"\nTreinando o modelo para prever o canal {canal_alvo} com hiperparâmetros: n_feat={n_feat}, lr={lr}, batch_size={batch_size}, epochs={epochs}\n")
        
        fold_results = []
        
        # K-Fold Cross-Validation
        for fold, (train_idx, val_idx) in enumerate(kfold.split(inputs_tensor)):
            print(f"Fold {fold+1}/{num_folds}")
            
            # Dividir os dados em treinamento e validação
            train_inputs = inputs_tensor[train_idx]
            train_targets = targets_tensor[train_idx]
            val_inputs = inputs_tensor[val_idx]
            val_targets = targets_tensor[val_idx]
            
            # Criar Dataset e DataLoader para treinamento
            train_dataset = torch.utils.data.TensorDataset(train_inputs, train_targets)
            train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            
            # Criar Dataset e DataLoader para validação
            val_dataset = torch.utils.data.TensorDataset(val_inputs, val_targets)
            val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            
            # Definir o modelo U-Net ajustado para os canais de EEG
            time_emb_dim = 128
            in_channels = len(canais_entrada) + 1  # Canais de entrada + canal alvo com ruído adicionado
            out_channels = 1  # Ruído previsto para o canal alvo
            model = UNet(in_channels=in_channels, out_channels=out_channels, n_feat=n_feat, time_emb_dim=time_emb_dim).to(device)
            
            # Otimizador e scheduler
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
            
            # Treinar o modelo
            train_diffusion_model(model, optimizer, scheduler, train_dataloader, timesteps, betas,
                                  sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, epochs=epochs)
            
            # Avaliação no conjunto de validação
            model.eval()
            val_losses = []
            with torch.no_grad():
                for batch_inputs, batch_targets in val_dataloader:
                    batch_inputs = batch_inputs.to(device)
                    batch_targets = batch_targets.to(device)
                    batch_size = batch_inputs.size(0)
                    t = torch.randint(0, timesteps, (batch_size,), device=device).long()

                    # Adicionar ruído aos alvos
                    noise = torch.randn_like(batch_targets)
                    sqrt_alphas_cumprod_t = sqrt_alphas_cumprod[t].view(batch_size, 1, 1)
                    sqrt_one_minus_alphas_cumprod_t = sqrt_one_minus_alphas_cumprod[t].view(batch_size, 1, 1)
                    x_t = sqrt_alphas_cumprod_t * batch_targets + sqrt_one_minus_alphas_cumprod_t * noise

                    # Concatenar entradas e x_t
                    model_input = torch.cat([batch_inputs, x_t], dim=1)

                    noise_pred = model(model_input, t)
                    loss = nn.MSELoss()(noise_pred, noise)
                    val_losses.append(loss.item())
            avg_val_loss = np.mean(val_losses)
            print(f"Val Loss: {avg_val_loss:.4f}")
            fold_results.append(avg_val_loss)
        
        # Média dos resultados nos folds
        mean_val_loss = np.mean(fold_results)
        std_val_loss = np.std(fold_results)
        print(f"Média da Val Loss nos folds: {mean_val_loss:.4f} ± {std_val_loss:.4f}")
        
        # Armazenar os resultados
        avaliacao_resultados.append({
            'Canal_Alvo': canal_alvo,
            'Canais_Entrada': ', '.join(canais_entrada),
            'n_feat': n_feat,
            'lr': lr,
            'batch_size': batch_size,
            'epochs': epochs,
            'Mean_Val_Loss': mean_val_loss,
            'Std_Val_Loss': std_val_loss
        })

# Converter os resultados em um DataFrame
avaliacao_df = pd.DataFrame(avaliacao_resultados)
print("\nResultados da Avaliação:")
print(avaliacao_df)

# Salvar os resultados da avaliação em um arquivo CSV
avaliacao_df.to_csv('avaliacao_resultados.csv', index=False)
print("Resultados da avaliação salvos em 'avaliacao_resultados.csv'")


## WGAN

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

# Configurações gerais
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Função de penalidade do gradiente para WGAN-GP
def gradient_penalty(discriminator, real_data, fake_data, cond, lambda_gp=10):
    batch_size = real_data.size(0)
    epsilon = torch.rand(batch_size, 1, device=device)
    epsilon = epsilon.expand_as(real_data)
    interpolated = epsilon * real_data + (1 - epsilon) * fake_data
    interpolated = interpolated.detach().requires_grad_(True)
    
    prob_interpolated = discriminator(interpolated, cond)
    
    gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                                    grad_outputs=torch.ones(prob_interpolated.size()).to(device),
                                    create_graph=True, retain_graph=True)[0]
    
    gradients = gradients.view(batch_size, -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_gp
    return gradient_penalty

# Gerador Condicional
class Generator(nn.Module):
    def __init__(self, z_dim, cond_dim, output_dim, n_features=64):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim + cond_dim, n_features * 4),
            nn.ReLU(),
            nn.Linear(n_features * 4, n_features * 8),
            nn.ReLU(),
            nn.Linear(n_features * 8, output_dim)
        )
    
    def forward(self, z, cond):
        # z: (batch_size, z_dim)
        # cond: (batch_size, cond_dim)
        x = torch.cat([z, cond], dim=1)
        return self.model(x)

# Discriminador Condicional
class Discriminator(nn.Module):
    def __init__(self, input_dim, cond_dim, n_features=64):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + cond_dim, n_features * 8),
            nn.ReLU(),
            nn.Linear(n_features * 8, n_features * 4),
            nn.ReLU(),
            nn.Linear(n_features * 4, 1)
        )
    
    def forward(self, x, cond):
        # x: (batch_size, input_dim)
        # cond: (batch_size, cond_dim)
        x = torch.cat([x, cond], dim=1)
        return self.model(x)

# Função de treinamento principal para WGAN-GP
def train_wgan_gp(generator, discriminator, dataloader, num_epochs=100, z_dim=100, lambda_gp=10):
    optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.9))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.9))
    
    for epoch in range(num_epochs):
        epoch_d_loss = 0
        epoch_g_loss = 0
        for batch_inputs, batch_targets in tqdm(dataloader, leave=False):
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            batch_size = batch_inputs.size(0)
            
            # Achatar as entradas e alvos
            cond = batch_inputs.reshape(batch_size, -1)  # Usar reshape em vez de view
            real_data = batch_targets.reshape(batch_size, -1)  # Usar reshape em vez de view
            
            # Treinamento do Discriminador
            for _ in range(5):
                z = torch.randn(batch_size, z_dim).to(device)
                fake_data = generator(z, cond)
                
                real_validity = discriminator(real_data, cond)
                fake_validity = discriminator(fake_data.detach(), cond)
                
                gp = gradient_penalty(discriminator, real_data, fake_data.detach(), cond)
                
                d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + gp
                
                optimizer_D.zero_grad()
                d_loss.backward()
                optimizer_D.step()
                epoch_d_loss += d_loss.item()
            
            # Treinamento do Gerador
            z = torch.randn(batch_size, z_dim).to(device)
            fake_data = generator(z, cond)
            g_loss = -torch.mean(discriminator(fake_data, cond))
            
            optimizer_G.zero_grad()
            g_loss.backward()
            optimizer_G.step()
            epoch_g_loss += g_loss.item()
        
        avg_d_loss = epoch_d_loss / len(dataloader)
        avg_g_loss = epoch_g_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {avg_d_loss:.4f}, G Loss: {avg_g_loss:.4f}")

# Função para gerar dados sintéticos usando o Gerador treinado
def generate_synthetic_data(generator, conditioning_inputs, z_dim=100):
    generator.eval()
    with torch.no_grad():
        batch_size = conditioning_inputs.size(0)
        z = torch.randn(batch_size, z_dim).to(device)
        synthetic_data = generator(z, conditioning_inputs)
    return synthetic_data.cpu().numpy()

# Carregar os dados de EEG utilizando pandas
eeg_df = pd.read_csv('bci_competition_3_v.csv')

# Lista de diretrizes para criar canais sintéticos
diretrizes = [
    # {'canal_alvo': 'F7', 'canais_entrada': ['FC5', 'F3']},
    # Adicione mais diretrizes conforme necessário
    # {'canal_alvo': 'AF3', 'canais_entrada': ['Fp1', 'F3']},
    # {'canal_alvo': 'AF4', 'canais_entrada': ['Fp2', 'F4']},
    # {'canal_alvo': 'F8', 'canais_entrada': ['T8', 'F4']},
    # {'canal_alvo': 'T7', 'canais_entrada': ['C3', 'CP1']},
    # {'canal_alvo': 'T8', 'canais_entrada': ['C4', 'CP2']},
    # {'canal_alvo': 'Fp1', 'canais_entrada': ['AF3', 'F3']},
    {'canal_alvo': 'Fp2', 'canais_entrada': ['AF4', 'F4']}
]

# Lista para armazenar as métricas de avaliação
avaliacao_resultados = []

# Iterar sobre cada diretriz
for diretriz in diretrizes:
    canal_alvo = diretriz['canal_alvo']
    canais_entrada = diretriz['canais_entrada']
    
    print(f"\nTreinando o modelo WGAN-GP para prever o canal {canal_alvo} usando {canais_entrada}\n")
    
    # Selecionar os dados dos canais de entrada e do canal alvo
    inputs = eeg_df[canais_entrada].values
    targets = eeg_df[[canal_alvo]].values

    # Calcular a média e desvio padrão dos canais de entrada e alvo
    mean_inputs = inputs.mean(axis=0)
    std_inputs = inputs.std(axis=0)
    inputs_normalizado = (inputs - mean_inputs) / std_inputs

    mean_target = targets.mean(axis=0)
    std_target = targets.std(axis=0)
    targets_normalizado = (targets - mean_target) / std_target

    # Garantir que inputs e targets tenham o mesmo número de amostras
    min_length = min(len(inputs_normalizado), len(targets_normalizado))
    inputs_normalizado = inputs_normalizado[:min_length]
    targets_normalizado = targets_normalizado[:min_length]

    # Preparação dos dados: Segmentar em sequências menores
    seq_length = 128
    num_samples = inputs_normalizado.shape[0]
    num_segments = num_samples // seq_length

    inputs_normalizado = inputs_normalizado[:num_segments * seq_length]
    targets_normalizado = targets_normalizado[:num_segments * seq_length]

    inputs_segments = inputs_normalizado.reshape(num_segments, seq_length, len(canais_entrada))
    targets_segments = targets_normalizado.reshape(num_segments, seq_length, 1)

    # Transpor para (num_segments, num_channels, seq_length)
    inputs_segments = inputs_segments.transpose(0, 2, 1)
    targets_segments = targets_segments.transpose(0, 2, 1)

    # Criar Dataset personalizado
    class EEGDataset(torch.utils.data.Dataset):
        def __init__(self, inputs, targets):
            self.inputs = torch.tensor(inputs, dtype=torch.float32)
            self.targets = torch.tensor(targets, dtype=torch.float32)

        def __len__(self):
            return len(self.inputs)

        def __getitem__(self, idx):
            x = self.inputs[idx].reshape(-1)  # Usar reshape em vez de view
            y = self.targets[idx].reshape(-1)  # Usar reshape em vez de view
            return x, y

    # Criar o dataset e dataloader
    dataset = EEGDataset(inputs_segments, targets_segments)
    batch_size = 32
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Definir as dimensões
    z_dim = 100
    cond_dim = inputs_segments.shape[1] * inputs_segments.shape[2]  # Número de canais de entrada * comprimento da sequência
    output_dim = targets_segments.shape[1] * targets_segments.shape[2]  # 1 * comprimento da sequência

    # Inicializar o Gerador e o Discriminador
    generator = Generator(z_dim=z_dim, cond_dim=cond_dim, output_dim=output_dim).to(device)
    discriminator = Discriminator(input_dim=output_dim, cond_dim=cond_dim).to(device)

    # Treinar o modelo WGAN-GP
    num_epochs = 100  # Ajuste conforme necessário
    train_wgan_gp(generator, discriminator, dataloader, num_epochs=num_epochs, z_dim=z_dim, lambda_gp=10)

    # Salvar o modelo treinado
    model_filename = f'generator_{canal_alvo}.pth'
    torch.save(generator.state_dict(), model_filename)
    print(f"Modelo do Gerador salvo como {model_filename}")

    # Gerar o canal sintético para todos os segmentos
    conditioning_inputs = torch.tensor(inputs_segments, dtype=torch.float32).to(device)
    conditioning_inputs_flat = conditioning_inputs.reshape(conditioning_inputs.size(0), -1)  # Usar reshape em vez de view

    # Gerar o canal sintético
    synthetic_data = generate_synthetic_data(generator, conditioning_inputs_flat, z_dim=z_dim)

    # Remodelar os dados sintéticos
    synthetic_data = synthetic_data.reshape(-1, 1, seq_length)

    # Desnormalizar os dados
    synthetic_data_desnormalizado = synthetic_data * std_target + mean_target  # Desnormalizar

    # Flatten the synthetic data
    synthetic_channel_flat = synthetic_data_desnormalizado.reshape(-1)

    # Criar uma nova coluna no eeg_df para o canal sintético
    column_name = f'Sintetico_{canal_alvo}'

    # Garantir que o comprimento corresponda aos índices em eeg_df
    eeg_df[column_name] = np.nan  # Inicializar a nova coluna com NaNs
    eeg_df.loc[:len(synthetic_channel_flat)-1, column_name] = synthetic_channel_flat

    # Calcular o MSE e a correlação usando todos os dados
    original_target = targets_normalizado.reshape(-1) * std_target + mean_target
    synthetic_target = synthetic_channel_flat

    mse = mean_squared_error(original_target[:len(synthetic_target)], synthetic_target)
    corr = np.corrcoef(original_target[:len(synthetic_target)], synthetic_target)[0, 1]
    print(f"MSE para {canal_alvo} = {mse:.4f}, Correlação = {corr:.4f}")

    # Armazenar as métricas de avaliação
    avaliacao_resultados.append({
        'Canal_Alvo': canal_alvo,
        'Canais_Entrada': ', '.join(canais_entrada),
        'MSE': mse,
        'Correlacao': corr
    })

# Converter os resultados em um DataFrame
avaliacao_df = pd.DataFrame(avaliacao_resultados)
print("\nResultados da Avaliação:")
print(avaliacao_df)

# Salvar os resultados da avaliação em um arquivo CSV
avaliacao_df.to_csv(f'avaliacao_resultados_wgan_{canal_alvo}.csv', index=False)
print(f"Resultados da avaliação salvos em 'avaliacao_resultados_wgan_{canal_alvo}.csv'")

# Salvar o eeg_df com os dados sintéticos
eeg_df.to_csv(f'eeg_com_sintetico_wgan_{canal_alvo}.csv', index=False)
print(f"Dados sintéticos adicionados e salvos em 'eeg_com_sintetico_wgan_{canal_alvo}.csv'")


## Implementação do GA c/ Modelo de Difusão

In [28]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Função para limpar valores inválidos em uma string numérica
def clean_numeric_values(value):
    # Remover caracteres indesejados como '.' e '-'
    cleaned_value = str(value).replace('.', '').replace('-', '')
    try:
        return float(cleaned_value)
    except ValueError:
        return np.nan  # Se não for possível converter, retorna NaN

# Carregar o dataset com o delimitador correto
eeg_df_syntetic = pd.read_csv('bci_competition_3_v_syntetic_modelo_difusao.csv', delimiter=';')

# Verificar as colunas disponíveis no dataset
print(eeg_df_syntetic.columns)

# Definir as colunas que são os canais no dataset (ajuste conforme necessário)
channels = eeg_df_syntetic.columns[4:]  # Ajuste o índice se necessário

# Aplicar a função de limpeza a todas as colunas de canais
eeg_df_syntetic[channels] = eeg_df_syntetic[channels].applymap(clean_numeric_values)

# Converter as colunas de canais para números (substituir valores inválidos por NaN)
eeg_df_syntetic[channels] = eeg_df_syntetic[channels].apply(pd.to_numeric, errors='coerce')

# Preencher valores NaN com a média das colunas
eeg_df_syntetic[channels] = eeg_df_syntetic[channels].fillna(eeg_df_syntetic[channels].mean())

# Definir variáveis independentes e a variável alvo
X = eeg_df_syntetic[channels]
y = eeg_df_syntetic['label']  # Assumindo que a coluna 'label' contém as classes, ajuste se necessário

# Dividir em treino e teste
X_train_combined, X_test_combined, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Configurações do Algoritmo Genético
num_channels = len(channels)

# Criar classes do GA
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Função para criar um indivíduo (seleção de canais)
def create_individual():
    individual = [random.randint(0, 1) for _ in range(num_channels)]
    while sum(individual) == 0:
        individual = [random.randint(0, 1) for _ in range(num_channels)]
    return creator.Individual(individual)

# Função para avaliar o indivíduo
def evaluate(individual):
    selected_channels = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_channels) == 0:
        return 0,
    X_train_selected = X_train_combined.iloc[:, selected_channels]
    X_test_selected = X_test_combined.iloc[:, selected_channels]
    classifier = KNeighborsClassifier(n_neighbors=5)
    classifier.fit(X_train_selected, y_train)
    y_pred = classifier.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy,

# Configurar a toolbox do GA
toolbox = base.Toolbox()
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Definir parâmetros do GA
population_size = 50
num_generations = 10
mutation_prob = 0.2
crossover_prob = 0.5

# Criar a população inicial
population = toolbox.population(n=population_size)

# Estatísticas
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("max", np.max)

# Executar o Algoritmo Genético
population, logbook = algorithms.eaSimple(population, toolbox,
                                          cxpb=crossover_prob,
                                          mutpb=mutation_prob,
                                          ngen=num_generations,
                                          stats=stats,
                                          verbose=True)

# Análise dos resultados
best_individual = tools.selBest(population, k=1)[0]
selected_channels_indices = [i for i, bit in enumerate(best_individual) if bit == 1]
selected_channels = [channels[i] for i in selected_channels_indices]

print(f"Canais selecionados: {selected_channels}")

# Identificar se canais sintéticos foram selecionados
channels_to_remove = ['AF3', 'AF4', 'F7', 'F8', 'Fp1', 'Fp2', 'T7', 'T8']  # Exemplos de canais sintéticos gerados
artificial_channels_selected = [ch for ch in channels_to_remove if ch in selected_channels]
print(f"Canais artificiais selecionados: {artificial_channels_selected}")


Index(['patient', 'time', 'label', 'epoch', 'Fp1', 'AF3', 'F7', 'F3', 'FC1',
       'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz',
       'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4',
       'F8', 'AF4', 'Fp2', 'Fz', 'Cz'],
      dtype='object')


/var/folders/xv/m10cj0cx4ls1q1q7t63fvykr0000gn/T/ipykernel_35967/1742261654.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  eeg_df_syntetic[channels] = eeg_df_syntetic[channels].applymap(clean_numeric_values)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg     	max     
0  	50    	0.344576	0.351121
1  	26    	0.345355	0.350685
2  	31    	0.346454	0.351693
3  	27    	0.348281	0.353191
4  	24    	0.349098	0.353191
5  	29    	0.349988	0.353191
6  	35    	0.350335	0.353436
7  	30    	0.350363	0.353436
8  	30    	0.351281	0.353436
9  	36    	0.351597	0.353436
10 	27    	0.352204	0.353436
Canais selecionados: ['Fp1', 'F7', 'T7', 'CP1', 'CP5', 'P7', 'P3', 'Pz', 'PO3', 'O1', 'O2', 'CP6', 'C4', 'T8', 'FC2', 'Fp2']
Canais artificiais selecionados: ['F7', 'Fp1', 'Fp2', 'T7', 'T8']


## Classificadores (Regressão Logistica, KNN, CNN e U-Net)

### Com Modelo de Difusão

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Conv1D, MaxPooling1D, Flatten, Input,
    UpSampling1D, concatenate, ZeroPadding1D, Cropping1D
)
from tensorflow.keras.utils import to_categorical

class EEGDataLoader:
    def __init__(self, original_file, synthetic_file, label_column='label'):
        self.original_file = original_file
        self.synthetic_file = synthetic_file
        self.label_column = label_column

    def load_data(self):
        # Carregar o dataset original
        df_original = pd.read_csv(self.original_file)
        # Carregar o dataset sintético com separador ';' e ajustando decimal e milhares
        df_sintetico = pd.read_csv(
            self.synthetic_file, sep=';', decimal=',', thousands='.'
        )
        # Remover linhas com valores ausentes
        df_original = df_original.dropna()
        df_sintetico = df_sintetico.dropna()
        return df_original, df_sintetico

    def prepare_features_and_labels(self, df):
        X = df.drop(columns=[self.label_column])
        y = df[self.label_column]
        return X, y

class EEGClassifier:
    def __init__(self, classifier, classifier_name):
        self.classifier = classifier
        self.classifier_name = classifier_name
        self.scaler = StandardScaler()
        self.is_keras_model = isinstance(classifier, tf.keras.Model)

    def train(self, X_train, y_train):
        if self.is_keras_model:
            # Escalonar os dados mantendo a forma
            nsamples, nx, ny = X_train.shape
            X_train_flat = X_train.reshape((nsamples, nx * ny))
            X_train_scaled = self.scaler.fit_transform(X_train_flat)
            X_train_scaled = X_train_scaled.reshape((nsamples, nx, ny))
            # Converter rótulos para categóricos
            num_classes = len(np.unique(y_train))
            y_train_categorical = to_categorical(y_train, num_classes=num_classes)
            # Compilar e treinar o modelo
            self.classifier.compile(
                optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
            )
            self.classifier.fit(
                X_train_scaled, y_train_categorical, epochs=10, batch_size=32, verbose=0
            )
        else:
            # Escalonar os dados normalmente
            X_train_scaled = self.scaler.fit_transform(X_train)
            self.classifier.fit(X_train_scaled, y_train)

    def predict(self, X_test):
        if self.is_keras_model:
            nsamples, nx, ny = X_test.shape
            X_test_flat = X_test.reshape((nsamples, nx * ny))
            X_test_scaled = self.scaler.transform(X_test_flat)
            X_test_scaled = X_test_scaled.reshape((nsamples, nx, ny))
            y_pred_proba = self.classifier.predict(X_test_scaled)
            y_pred = np.argmax(y_pred_proba, axis=1)
            return y_pred
        else:
            X_test_scaled = self.scaler.transform(X_test)
            return self.classifier.predict(X_test_scaled)

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        acuracia = accuracy_score(y_test, y_pred)
        matriz_confusao = confusion_matrix(y_test, y_pred)
        precisao = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        return {
            'Acurácia': acuracia,
            'Precisão': precisao,
            'Recall': recall,
            'F1-Score': f1,
            'Matriz de Confusão': matriz_confusao
        }

# Funções para criar os modelos CNN e U-Net
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2, padding='same'))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, padding='same'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def create_unet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    # Encoder
    conv1 = Conv1D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling1D(pool_size=2, padding='same')(conv1)

    conv2 = Conv1D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=2, padding='same')(conv2)

    # Bottleneck
    conv3 = Conv1D(256, 3, activation='relu', padding='same')(pool2)

    # Decoder
    up4 = UpSampling1D(size=2)(conv3)
    # Ajuste de dimensões antes da concatenação
    diff4 = conv2.shape[1] - up4.shape[1]
    if diff4 > 0:
        up4 = ZeroPadding1D(padding=(0, diff4))(up4)
    elif diff4 < 0:
        up4 = Cropping1D(cropping=(0, -diff4))(up4)
    merge4 = concatenate([conv2, up4], axis=-1)
    conv4 = Conv1D(128, 3, activation='relu', padding='same')(merge4)

    up5 = UpSampling1D(size=2)(conv4)
    # Ajuste de dimensões antes da concatenação
    diff5 = conv1.shape[1] - up5.shape[1]
    if diff5 > 0:
        up5 = ZeroPadding1D(padding=(0, diff5))(up5)
    elif diff5 < 0:
        up5 = Cropping1D(cropping=(0, -diff5))(up5)
    merge5 = concatenate([conv1, up5], axis=-1)
    conv5 = Conv1D(64, 3, activation='relu', padding='same')(merge5)

    flat = Flatten()(conv5)
    outputs = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Uso das classes e funções
if __name__ == '__main__':
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier

    # Inicializar o carregador de dados
    data_loader = EEGDataLoader(
        original_file='bci_competition_3_v.csv',
        synthetic_file='bci_competition_3_v_syntetic_modelo_difusao.csv',
        label_column='label'
    )
    df_original, df_sintetico = data_loader.load_data()

    # Preparar características e rótulos para os dados originais
    X_original, y_original = data_loader.prepare_features_and_labels(df_original)
    # Codificar os rótulos
    le = LabelEncoder()
    y_original_encoded = le.fit_transform(y_original)

    # Dividir os dados em conjuntos de treinamento e teste
    X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(
        X_original, y_original_encoded, test_size=0.2, random_state=42
    )

    # Preparar características e rótulos para os dados sintéticos
    X_sintetico, y_sintetico = data_loader.prepare_features_and_labels(df_sintetico)
    y_sintetico_encoded = le.transform(y_sintetico)
    X_train_sint, X_test_sint, y_train_sint, y_test_sint = train_test_split(
        X_sintetico, y_sintetico_encoded, test_size=0.2, random_state=42
    )

    # Determinar o número de classes e input_shape
    num_classes = len(le.classes_)
    input_shape = (X_train_orig.shape[1], 1)  # Para Conv1D, input_shape é (timesteps, features)

    # Reshape dos dados para modelos Keras
    X_train_orig_reshaped = X_train_orig.values.reshape(-1, X_train_orig.shape[1], 1)
    X_test_orig_reshaped = X_test_orig.values.reshape(-1, X_test_orig.shape[1], 1)

    X_train_sint_reshaped = X_train_sint.values.reshape(-1, X_train_sint.shape[1], 1)
    X_test_sint_reshaped = X_test_sint.values.reshape(-1, X_test_sint.shape[1], 1)

    # Lista de classificadores para testar
    classifiers = [
        (LogisticRegression(max_iter=1000), 'Regressão Logística'),
        (KNeighborsClassifier(n_neighbors=5), 'KNN'),
        (create_cnn_model(input_shape=input_shape, num_classes=num_classes), 'CNN'),
        (create_unet_model(input_shape=input_shape, num_classes=num_classes), 'U-Net')
    ]

    # Lista para armazenar os resultados
    lista_resultados = []

    # Avaliar cada classificador nos dois datasets
    for clf, clf_name in classifiers:
        print(f"\nTreinando e avaliando o classificador: {clf_name}")

        # Dados Originais
        if clf_name in ['CNN', 'U-Net']:
            eeg_clf_orig = EEGClassifier(clf, clf_name)
            eeg_clf_orig.train(X_train_orig_reshaped, y_train_orig)
            metrics_orig = eeg_clf_orig.evaluate(X_test_orig_reshaped, y_test_orig)
        else:
            eeg_clf_orig = EEGClassifier(clf, clf_name)
            eeg_clf_orig.train(X_train_orig, y_train_orig)
            metrics_orig = eeg_clf_orig.evaluate(X_test_orig, y_test_orig)

        lista_resultados.append({
            'Classificador': clf_name,
            'Dataset': 'Original',
            'Acurácia': metrics_orig['Acurácia'],
            'Precisão': metrics_orig['Precisão'],
            'Recall': metrics_orig['Recall'],
            'F1-Score': metrics_orig['F1-Score']
        })
        print(f"\n{clf_name} - Dados Originais")
        print(f"Acurácia: {metrics_orig['Acurácia']:.4f}")
        print(f"Precisão: {metrics_orig['Precisão']:.4f}")
        print(f"Recall: {metrics_orig['Recall']:.4f}")
        print(f"F1-Score: {metrics_orig['F1-Score']:.4f}")
        print(f"Matriz de Confusão:\n{metrics_orig['Matriz de Confusão']}")

        # Dados Sintéticos
        if clf_name in ['CNN', 'U-Net']:
            if clf_name == 'CNN':
                clf_sint = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
            else:
                clf_sint = create_unet_model(input_shape=input_shape, num_classes=num_classes)
            eeg_clf_sint = EEGClassifier(clf_sint, clf_name)
            eeg_clf_sint.train(X_train_sint_reshaped, y_train_sint)
            metrics_sint = eeg_clf_sint.evaluate(X_test_sint_reshaped, y_test_sint)
        else:
            eeg_clf_sint = EEGClassifier(clf, clf_name)
            eeg_clf_sint.train(X_train_sint, y_train_sint)
            metrics_sint = eeg_clf_sint.evaluate(X_test_sint, y_test_sint)

        lista_resultados.append({
            'Classificador': clf_name,
            'Dataset': 'Sintético',
            'Acurácia': metrics_sint['Acurácia'],
            'Precisão': metrics_sint['Precisão'],
            'Recall': metrics_sint['Recall'],
            'F1-Score': metrics_sint['F1-Score']
        })
        print(f"\n{clf_name} - Dados Sintéticos")
        print(f"Acurácia: {metrics_sint['Acurácia']:.4f}")
        print(f"Precisão: {metrics_sint['Precisão']:.4f}")
        print(f"Recall: {metrics_sint['Recall']:.4f}")
        print(f"F1-Score: {metrics_sint['F1-Score']:.4f}")
        print(f"Matriz de Confusão:\n{metrics_sint['Matriz de Confusão']}")

    # Converter a lista de resultados em um DataFrame
    resultados = pd.DataFrame(lista_resultados)

    # Exibir a tabela de resultados
    print("\nTabela de Resultados:")
    print(resultados)

    # Opcional: Salvar os resultados em um arquivo CSV
    resultados.to_csv('resultados_classificacao.csv', index=False)
    print("\nOs resultados foram salvos em 'resultados_classificacao.csv'")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Treinando e avaliando o classificador: Regressão Logística

Regressão Logística - Dados Originais
Acurácia: 0.3714
Precisão: 0.1845
Recall: 0.3714
F1-Score: 0.2295
Matriz de Confusão:
[[   0    1 6425]
 [   0   92 7834]
 [   0 1125 8997]]

Regressão Logística - Dados Sintéticos
Acurácia: 0.3924
Precisão: 0.3287
Recall: 0.3924
F1-Score: 0.2888
Matriz de Confusão:
[[ 162  387 5877]
 [ 326  774 6826]
 [ 432 1023 8667]]

Treinando e avaliando o classificador: KNN

KNN - Dados Originais
Acurácia: 0.9307
Precisão: 0.9320
Recall: 0.9307
F1-Score: 0.9309
Matriz de Confusão:
[[6085  196  145]
 [ 331 7402  193]
 [ 379  452 9291]]

KNN - Dados Sintéticos
Acurácia: 0.4058
Precisão: 0.4208
Recall: 0.4058
F1-Score: 0.4088
Matriz de Confusão:
[[2847 1895 1684]
 [2663 2987 2276]
 [3056 2968 4098]]

Treinando e avaliando o classificador: CNN
765/765 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step

CNN - Dados Originais
Acurácia: 0.9889
Precisão: 0.9890
Recall: 0.9889
F1-Score: 0.9889
Matriz de Confusão:
[[ 6300   

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


765/765 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step

CNN - Dados Sintéticos
Acurácia: 0.9572
Precisão: 0.9575
Recall: 0.9572
F1-Score: 0.9571
Matriz de Confusão:
[[5971  176  279]
 [   9 7690  227]
 [ 125  232 9765]]

Treinando e avaliando o classificador: U-Net
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

U-Net - Dados Originais
Acurácia: 0.9879
Precisão: 0.9879
Recall: 0.9879
F1-Score: 0.9879
Matriz de Confusão:
[[6401   16    9]
 [  34 7852   40]
 [  77  121 9924]]
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

U-Net - Dados Sintéticos
Acurácia: 0.9631
Precisão: 0.9633
Recall: 0.9631
F1-Score: 0.9631
Matriz de Confusão:
[[6141  114  171]
 [  69 7574  283]
 [  56  210 9856]]

Tabela de Resultados:
         Classificador    Dataset  Acurácia  Precisão    Recall  F1-Score
0  Regressão Logística   Original  0.371374  0.184463  0.371374  0.229478
1  Regressão Logística  Sintético  0.392376  0.328742  0.392376  0.288814
2                  KNN   Original  0.930702  0.931978  0.930702  0.930891
3           

### Com WGANs

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Conv1D, MaxPooling1D, Flatten, Input,
    UpSampling1D, concatenate, ZeroPadding1D, Cropping1D
)
from tensorflow.keras.utils import to_categorical

class EEGDataLoader:
    def __init__(self, original_file, synthetic_file, label_column='label'):
        self.original_file = original_file
        self.synthetic_file = synthetic_file
        self.label_column = label_column

    def load_data(self):
        # Carregar o dataset original
        df_original = pd.read_csv(self.original_file)
        # Carregar o dataset sintético com separador ';' e ajustando decimal e milhares
        df_sintetico = pd.read_csv(
            self.synthetic_file, sep=';', decimal=',', thousands='.'
        )
        # Remover linhas com valores ausentes
        df_original = df_original.dropna()
        df_sintetico = df_sintetico.dropna()
        return df_original, df_sintetico

    def prepare_features_and_labels(self, df):
        X = df.drop(columns=[self.label_column])
        y = df[self.label_column]
        return X, y

class EEGClassifier:
    def __init__(self, classifier, classifier_name):
        self.classifier = classifier
        self.classifier_name = classifier_name
        self.scaler = StandardScaler()
        self.is_keras_model = isinstance(classifier, tf.keras.Model)

    def train(self, X_train, y_train):
        if self.is_keras_model:
            # Escalonar os dados mantendo a forma
            nsamples, nx, ny = X_train.shape
            X_train_flat = X_train.reshape((nsamples, nx * ny))
            X_train_scaled = self.scaler.fit_transform(X_train_flat)
            X_train_scaled = X_train_scaled.reshape((nsamples, nx, ny))
            # Converter rótulos para categóricos
            num_classes = len(np.unique(y_train))
            y_train_categorical = to_categorical(y_train, num_classes=num_classes)
            # Compilar e treinar o modelo
            self.classifier.compile(
                optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
            )
            self.classifier.fit(
                X_train_scaled, y_train_categorical, epochs=10, batch_size=32, verbose=0
            )
        else:
            # Escalonar os dados normalmente
            X_train_scaled = self.scaler.fit_transform(X_train)
            self.classifier.fit(X_train_scaled, y_train)

    def predict(self, X_test):
        if self.is_keras_model:
            nsamples, nx, ny = X_test.shape
            X_test_flat = X_test.reshape((nsamples, nx * ny))
            X_test_scaled = self.scaler.transform(X_test_flat)
            X_test_scaled = X_test_scaled.reshape((nsamples, nx, ny))
            y_pred_proba = self.classifier.predict(X_test_scaled)
            y_pred = np.argmax(y_pred_proba, axis=1)
            return y_pred
        else:
            X_test_scaled = self.scaler.transform(X_test)
            return self.classifier.predict(X_test_scaled)

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        acuracia = accuracy_score(y_test, y_pred)
        matriz_confusao = confusion_matrix(y_test, y_pred)
        precisao = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        return {
            'Acurácia': acuracia,
            'Precisão': precisao,
            'Recall': recall,
            'F1-Score': f1,
            'Matriz de Confusão': matriz_confusao
        }

# Funções para criar os modelos CNN e U-Net
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2, padding='same'))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, padding='same'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def create_unet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    # Encoder
    conv1 = Conv1D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling1D(pool_size=2, padding='same')(conv1)

    conv2 = Conv1D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling1D(pool_size=2, padding='same')(conv2)

    # Bottleneck
    conv3 = Conv1D(256, 3, activation='relu', padding='same')(pool2)

    # Decoder
    up4 = UpSampling1D(size=2)(conv3)
    # Ajuste de dimensões antes da concatenação
    diff4 = conv2.shape[1] - up4.shape[1]
    if diff4 > 0:
        up4 = ZeroPadding1D(padding=(0, diff4))(up4)
    elif diff4 < 0:
        up4 = Cropping1D(cropping=(0, -diff4))(up4)
    merge4 = concatenate([conv2, up4], axis=-1)
    conv4 = Conv1D(128, 3, activation='relu', padding='same')(merge4)

    up5 = UpSampling1D(size=2)(conv4)
    # Ajuste de dimensões antes da concatenação
    diff5 = conv1.shape[1] - up5.shape[1]
    if diff5 > 0:
        up5 = ZeroPadding1D(padding=(0, diff5))(up5)
    elif diff5 < 0:
        up5 = Cropping1D(cropping=(0, -diff5))(up5)
    merge5 = concatenate([conv1, up5], axis=-1)
    conv5 = Conv1D(64, 3, activation='relu', padding='same')(merge5)

    flat = Flatten()(conv5)
    outputs = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Uso das classes e funções
if __name__ == '__main__':
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier

    # Inicializar o carregador de dados
    data_loader = EEGDataLoader(
        original_file='bci_competition_3_v.csv',
        synthetic_file='bci_competition_3_v_syntetic_wgan.csv',
        label_column='label'
    )
    df_original, df_sintetico = data_loader.load_data()

    # Preparar características e rótulos para os dados originais
    X_original, y_original = data_loader.prepare_features_and_labels(df_original)
    # Codificar os rótulos
    le = LabelEncoder()
    y_original_encoded = le.fit_transform(y_original)

    # Dividir os dados em conjuntos de treinamento e teste
    X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(
        X_original, y_original_encoded, test_size=0.2, random_state=42
    )

    # Preparar características e rótulos para os dados sintéticos
    X_sintetico, y_sintetico = data_loader.prepare_features_and_labels(df_sintetico)
    y_sintetico_encoded = le.transform(y_sintetico)
    X_train_sint, X_test_sint, y_train_sint, y_test_sint = train_test_split(
        X_sintetico, y_sintetico_encoded, test_size=0.2, random_state=42
    )

    # Determinar o número de classes e input_shape
    num_classes = len(le.classes_)
    input_shape = (X_train_orig.shape[1], 1)  # Para Conv1D, input_shape é (timesteps, features)

    # Reshape dos dados para modelos Keras
    X_train_orig_reshaped = X_train_orig.values.reshape(-1, X_train_orig.shape[1], 1)
    X_test_orig_reshaped = X_test_orig.values.reshape(-1, X_test_orig.shape[1], 1)

    X_train_sint_reshaped = X_train_sint.values.reshape(-1, X_train_sint.shape[1], 1)
    X_test_sint_reshaped = X_test_sint.values.reshape(-1, X_test_sint.shape[1], 1)

    # Lista de classificadores para testar
    classifiers = [
        (LogisticRegression(max_iter=1000), 'Regressão Logística'),
        (KNeighborsClassifier(n_neighbors=5), 'KNN'),
        (create_cnn_model(input_shape=input_shape, num_classes=num_classes), 'CNN'),
        (create_unet_model(input_shape=input_shape, num_classes=num_classes), 'U-Net')
    ]

    # Lista para armazenar os resultados
    lista_resultados = []

    # Avaliar cada classificador nos dois datasets
    for clf, clf_name in classifiers:
        print(f"\nTreinando e avaliando o classificador: {clf_name}")

        # Dados Originais
        if clf_name in ['CNN', 'U-Net']:
            eeg_clf_orig = EEGClassifier(clf, clf_name)
            eeg_clf_orig.train(X_train_orig_reshaped, y_train_orig)
            metrics_orig = eeg_clf_orig.evaluate(X_test_orig_reshaped, y_test_orig)
        else:
            eeg_clf_orig = EEGClassifier(clf, clf_name)
            eeg_clf_orig.train(X_train_orig, y_train_orig)
            metrics_orig = eeg_clf_orig.evaluate(X_test_orig, y_test_orig)

        lista_resultados.append({
            'Classificador': clf_name,
            'Dataset': 'Original',
            'Acurácia': metrics_orig['Acurácia'],
            'Precisão': metrics_orig['Precisão'],
            'Recall': metrics_orig['Recall'],
            'F1-Score': metrics_orig['F1-Score']
        })
        print(f"\n{clf_name} - Dados Originais")
        print(f"Acurácia: {metrics_orig['Acurácia']:.4f}")
        print(f"Precisão: {metrics_orig['Precisão']:.4f}")
        print(f"Recall: {metrics_orig['Recall']:.4f}")
        print(f"F1-Score: {metrics_orig['F1-Score']:.4f}")
        print(f"Matriz de Confusão:\n{metrics_orig['Matriz de Confusão']}")

        # Dados Sintéticos
        if clf_name in ['CNN', 'U-Net']:
            if clf_name == 'CNN':
                clf_sint = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
            else:
                clf_sint = create_unet_model(input_shape=input_shape, num_classes=num_classes)
            eeg_clf_sint = EEGClassifier(clf_sint, clf_name)
            eeg_clf_sint.train(X_train_sint_reshaped, y_train_sint)
            metrics_sint = eeg_clf_sint.evaluate(X_test_sint_reshaped, y_test_sint)
        else:
            eeg_clf_sint = EEGClassifier(clf, clf_name)
            eeg_clf_sint.train(X_train_sint, y_train_sint)
            metrics_sint = eeg_clf_sint.evaluate(X_test_sint, y_test_sint)

        lista_resultados.append({
            'Classificador': clf_name,
            'Dataset': 'Sintético',
            'Acurácia': metrics_sint['Acurácia'],
            'Precisão': metrics_sint['Precisão'],
            'Recall': metrics_sint['Recall'],
            'F1-Score': metrics_sint['F1-Score']
        })
        print(f"\n{clf_name} - Dados Sintéticos")
        print(f"Acurácia: {metrics_sint['Acurácia']:.4f}")
        print(f"Precisão: {metrics_sint['Precisão']:.4f}")
        print(f"Recall: {metrics_sint['Recall']:.4f}")
        print(f"F1-Score: {metrics_sint['F1-Score']:.4f}")
        print(f"Matriz de Confusão:\n{metrics_sint['Matriz de Confusão']}")

    # Converter a lista de resultados em um DataFrame
    resultados = pd.DataFrame(lista_resultados)

    # Exibir a tabela de resultados
    print("\nTabela de Resultados:")
    print(resultados)

    # Opcional: Salvar os resultados em um arquivo CSV
    resultados.to_csv('resultados_classificacao_wgan.csv', index=False)
    print("\nOs resultados foram salvos em 'resultados_classificacao_wgan.csv'")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Treinando e avaliando o classificador: Regressão Logística

Regressão Logística - Dados Originais
Acurácia: 0.3714
Precisão: 0.1845
Recall: 0.3714
F1-Score: 0.2295
Matriz de Confusão:
[[   0    1 6425]
 [   0   92 7834]
 [   0 1125 8997]]

Regressão Logística - Dados Sintéticos
Acurácia: 0.3982
Precisão: 0.2947
Recall: 0.3982
F1-Score: 0.2782
Matriz de Confusão:
[[   6  335 6085]
 [  21  741 7164]
 [  87 1037 8998]]

Treinando e avaliando o classificador: KNN

KNN - Dados Originais
Acurácia: 0.9307
Precisão: 0.9320
Recall: 0.9307
F1-Score: 0.9309
Matriz de Confusão:
[[6085  196  145]
 [ 331 7402  193]
 [ 379  452 9291]]

KNN - Dados Sintéticos
Acurácia: 0.4148
Precisão: 0.4302
Recall: 0.4148
F1-Score: 0.4175
Matriz de Confusão:
[[2973 1841 1612]
 [2644 3059 2223]
 [3008 2993 4121]]

Treinando e avaliando o classificador: CNN
765/765 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step

CNN - Dados Originais
Acurácia: 0.9881
Precisão: 0.9881
Recall: 0.9881
F1-Score: 0.9881
Matriz de Confusão:
[[ 6361   

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


765/765 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step

CNN - Dados Sintéticos
Acurácia: 0.9527
Precisão: 0.9534
Recall: 0.9527
F1-Score: 0.9527
Matriz de Confusão:
[[6311   11  104]
 [ 379 7316  231]
 [ 153  279 9690]]

Treinando e avaliando o classificador: U-Net
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

U-Net - Dados Originais
Acurácia: 0.9888
Precisão: 0.9888
Recall: 0.9888
F1-Score: 0.9888
Matriz de Confusão:
[[6352   35   39]
 [  13 7878   35]
 [  78   75 9969]]
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

U-Net - Dados Sintéticos
Acurácia: 0.9514
Precisão: 0.9521
Recall: 0.9514
F1-Score: 0.9514
Matriz de Confusão:
[[6097   28  301]
 [ 122 7364  440]
 [  47  251 9824]]

Tabela de Resultados:
         Classificador    Dataset  Acurácia  Precisão    Recall  F1-Score
0  Regressão Logística   Original  0.371374  0.184463  0.371374  0.229478
1  Regressão Logística  Sintético  0.398178  0.294667  0.398178  0.278227
2                  KNN   Original  0.930702  0.931978  0.930702  0.930891
3           